# Context-Aware Step Counting
This project notebook implements a step-counting algorithm that adjusts based on context (walking, jogging, stairs).
The following steps are followed:

1. **Problem Definition**: Justify the need for context-aware step counting.
2. **Sensor Selection**: Use accelerometer and gyroscope data.
3. **Data Collection**: Annotate and preprocess.
4. **Feature Engineering**: Extract meaningful features for classification.
5. **Classification**: Train models to detect context and adapt step-counting parameters.
6. **Visualization and Results**: Analyze signals and evaluate accuracy.

Each step is modular and reusable.

## Step 1: Problem Definition - Baseline Step Counting
Here, we demonstrate the limitations of a standard step-counting algorithm in different contexts.

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# Configure plots
plt.rcParams['figure.figsize'] = [9, 6]
plt.rcParams['axes.grid'] = True

# Helper function: Preprocess accelerometer data (magnitude calculation)
def calc_magnitude(df, x_col='x', y_col='y', z_col='z'):
    """Calculate acceleration magnitude and remove gravity effect."""
    df['accel_mag'] = np.sqrt(df[x_col]**2 + df[y_col]**2 + df[z_col]**2)
    df['accel_mag'] -= df['accel_mag'].mean()
    return df

# Helper function: Apply low-pass filter to reduce noise
def remove_noise(df, cutoff=5, sample_rate=50):
    """Apply a low-pass Butterworth filter to the signal."""
    nyquist = 0.5 * sample_rate
    normalized_cutoff = cutoff / nyquist
    b, a = butter(2, normalized_cutoff, btype='low', analog=False)
    df['filtered_accel_mag'] = filtfilt(b, a, df['accel_mag'])
    return df

# Load baseline data (walking data as an example)
data = pd.read_csv('./data/Baseline/walking.csv')
data = calc_magnitude(data)
data = remove_noise(data)

# Visualize filtered signal
plt.plot(data['filtered_accel_mag'][:1000])
plt.title('Filtered Acceleration Magnitude (Walking)')
plt.xlabel('Time (samples)')
plt.ylabel('Magnitude')
plt.show()

# Example: Evaluate step counting using peak detection
peaks, _ = find_peaks(data['filtered_accel_mag'], height=0.5, distance=30)
print(f"Detected steps: {len(peaks)}")

## Step 2: Sensor Selection
- Core sensors: Accelerometer, Gyroscope
- (Optional) Barometer for altitude detection

No code here, but ensure your devices support these sensors and collect the necessary data.

## Step 3: Data Collection and Labeling
Collect and preprocess annotated data for multiple activities.

In [2]:
# Transform collected data timestamps to datetime
def transform_time_to_datetime(root_dir):
    """Convert raw timestamps to datetime for all collected data."""
    for activity in os.listdir(root_dir):
        activity_dir = os.path.join(root_dir, activity)
        for file in glob.glob(f"{activity_dir}/*.csv"):
            df = pd.read_csv(file)
            df['time'] = pd.to_datetime(df['time'], unit='ns')
            df.to_csv(file, index=False)

# Example usage:
transform_time_to_datetime('./data/MyData')

## Step 4: Feature Engineering
Extract statistical and domain-specific features from the preprocessed data.

In [3]:
# Feature extraction (e.g., mean, std, quantiles)
def extract_features(window):
    """Extract features from a data window."""
    features = {
        'mean': window['filtered_accel_mag'].mean(),
        'std': window['filtered_accel_mag'].std(),
        'max': window['filtered_accel_mag'].max(),
        'min': window['filtered_accel_mag'].min()
    }
    return features

# Apply sliding window for feature extraction
def sliding_window_features(data, window_size=50):
    """Extract features using a sliding window."""
    features_list = []
    for start in range(0, len(data) - window_size + 1, window_size):
        window = data.iloc[start:start + window_size]
        features = extract_features(window)
        features_list.append(features)
    return pd.DataFrame(features_list)

# Example:
features = sliding_window_features(data)
features.head()

## Step 5: Classification
Train a model to classify activity contexts and adjust step-counting parameters.

In [4]:
# Train a classifier for context detection
X = features.drop(columns=['label'])  # Feature columns
y = features['label']                # Labels

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate performance
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

## Step 6: Dynamic Step Counting
Adjust step-counting parameters dynamically based on the detected context.

In [5]:
# Dynamic step counting based on context
def dynamic_step_counting(data, context):
    """Adjust step-counting parameters based on context."""
    params = {
        'walking': {'height': 0.5, 'distance': 30},
        'jogging': {'height': 1.0, 'distance': 20},
        'stairs': {'height': 0.7, 'distance': 25}
    }
    context_params = params.get(context, {'height': 0.5, 'distance': 30})
    peaks, _ = find_peaks(data['filtered_accel_mag'], 
                          height=context_params['height'], 
                          distance=context_params['distance'])
    return len(peaks)

# Example:
context = 'walking'  # Assume detected context
steps = dynamic_step_counting(data, context)
print(f"Detected steps in {context}: {steps}")

## Final Steps: Visualization and Reporting
- Visualize signals, classifications, and step detection accuracy.
- Generate confusion matrices and feature importance rankings.